In [3]:
import numpy as np

from modules.zapocet import * 


In [4]:
FI = 300  # mm
TW = 10  # mm
H = 50  # m
M = 1000  # kg
q = 1.5 # sucinitel duktility
ag = 0.63 # spickove zrychlenie
tvary = 3 # pocet tvarov
hmoty = 4 # pocet hmot
polozie = "C"
typ_spektra = 2

In [5]:
f = vlastne_frekvencie([0.071, 0.436, 1.2])
print(f'f1 = {f[0]} hz \nf2 = {f[1]} hz \nf3 = {f[2]} hz')

f1 = 0.071 hz 
f2 = 0.436 hz 
f3 = 1.2 hz


In [4]:
omega = uhlove_frekvencie(f)
print(f'omega1 = {omega[0]:.3f} rad/s \nomega2 = {omega[1]:.3f} rad/s \nomega3 = {omega[2]:.3f} rad/s')

omega1 = 0.446 rad/s 
omega2 = 2.739 rad/s 
omega3 = 7.540 rad/s


In [5]:
T = vlastne_periody(f)
print(f'T1 = {T[0]:.3f} s \nT2 = {T[1]:.3f} s \nT3 = {T[2]:.3f} s')

T1 = 14.085 s 
T2 = 2.294 s 
T3 = 0.833 s


Normovane poradnice vlastnych tvarov

In [6]:
# 1. poradnice vlastny tvar
por_1 = [0.09379, 0.33105, 0.64964, 1.00000]

# 2. poradnice vlastny tvar
por_2 = [0.51996, 1.00000, 0.46954, -0.89641]

# 3. poradnice vlastny tvar
por_3 = [-0.94157, -0.29134, 0.89985, -0.52332]

In [7]:
poradnice = np.array([por_1, por_2, por_3])
poradnice

array([[ 0.09379,  0.33105,  0.64964,  1.     ],
       [ 0.51996,  1.     ,  0.46954, -0.89641],
       [-0.94157, -0.29134,  0.89985, -0.52332]])

### Navrhove spektrum

In [20]:
parametre = spektrum_parametre("C", typ_spektra)
parametre

{'S': 1.5, 'T_B': 0.1, 'T_C': 0.25, 'T_D': 1.2}

In [21]:
def spektrum(ag, q, periody):
    Sd = []
    for perioda in periody:
        Sd.append(sd(parametre, perioda, ag, q))
    print(f'Sd1 = {Sd[0]:.3f} \nSd2 = {Sd[1]:.3f} \nSd3 = {Sd[2]:.3f}')
    return Sd

In [22]:
Sd = spektrum(ag, q, T)

Sd1 = 0.126 
Sd2 = 0.126 
Sd3 = 0.473


In [23]:
def ni(poradnice, M, stlpec, riadok):
    i = stlpec
    j = riadok

    poradnice_sum = np.sum(poradnice, axis=1)
    citatel = poradnice_sum[j]*M*poradnice[j][i]
    temp = np.array(poradnice[j])**2*M
    menovatel = np.sum(temp)

    return citatel/menovatel

In [24]:
ni_ = []
for i in range(hmoty):
    stlpce = []
    for j in range(tvary):
        stlpce.append(ni(poradnice, M, stlpec=i, riadok=j))
    ni_.append(stlpce)

In [25]:
ni_ = np.array(ni_).T
ni_

array([[ 0.12630654,  0.44582345,  0.87486708,  1.34669521],
       [ 0.24771999,  0.47642125,  0.22369883, -0.42706877],
       [ 0.39237526,  0.12140851, -0.37498952,  0.21808025]])

In [26]:
def odozva_preTvar(ni, M, Sd, ag, tvar):
    return ni[tvar]*ag*M*Sd[tvar]

In [27]:
odozvy = []
for i in range(tvary):
    temp = odozva_preTvar(ni_, M, Sd, ag, tvar=i)
    print(f'{i+1}. vlastny tvar \nR{i+1} = {temp}')
    odozvy.append(temp)
odozvy = np.array(odozvy)

1. vlastny tvar 
R1 = [ 10.02621344  35.38946539  69.44694848 106.90066572]
2. vlastny tvar 
R2 = [ 19.66401294  37.8183186   17.75721331 -33.90071897]
3. vlastny tvar 
R3 = [ 116.80030556   36.14027743 -111.62500394   64.91703846]


In [28]:
# bulharska_konstanta_1 = 8.37698
# bulharska_konstanta_2 = 0.054785

In [29]:
# R1 = odozvy[0]*bulharska_konstanta_1
# R2 = odozvy[1]
# R3 = odozvy[2]*bulharska_konstanta_2

In [30]:
for i in range(hmoty):
    V = np.sqrt(np.sum(odozvy[0][i:hmoty])**2+np.sum(odozvy[1][i:hmoty])**2+np.sum(odozvy[2][i:hmoty])**2)
    print(f'V{i+1} = {V:.3f} N')

V1 = 249.346 N
V2 = 213.106 N
V3 = 183.141 N
V4 = 129.581 N


In [31]:
ramena = H_vector(H)
print(f'r1 = {ramena[0]} m \nr2 = {ramena[1]} m \nr3 = {ramena[2]} m')

r1 = 12.5 m 
r2 = 25.0 m 
r3 = 37.5 m


In [32]:
def moment_preklopenia(M, ag, Sd, ni_, ramena, stlpec, riadok):
    i = stlpec
    j = riadok

    moment = M*ag*Sd[i]*ni_[i][j]*ramena[j]
    return moment

In [33]:
momenty_preklopenia = []
for j in range(hmoty):
    print(f'uzol {j+1}:')
    moment_v_uzle = []
    for i in range(tvary):
        temp = moment_preklopenia(M, ag, Sd, ni_, ramena, stlpec=i, riadok=j)
        print(f'Tvar{i+1} - M = {temp:.3f} Nm')
        moment_v_uzle.append(temp)
    momenty_preklopenia.append(moment_v_uzle)

uzol 1:
Tvar1 - M = 125.328 Nm
Tvar2 - M = 245.800 Nm
Tvar3 - M = 1460.004 Nm
uzol 2:
Tvar1 - M = 884.737 Nm
Tvar2 - M = 945.458 Nm
Tvar3 - M = 903.507 Nm
uzol 3:
Tvar1 - M = 2604.261 Nm
Tvar2 - M = 665.895 Nm
Tvar3 - M = -4185.938 Nm
uzol 4:
Tvar1 - M = 5345.033 Nm
Tvar2 - M = -1695.036 Nm
Tvar3 - M = 3245.852 Nm


In [34]:
def ohybove_momenty(odozvy, hmoty, ramena, tvary):
    for i in range(hmoty):
        odmocnit = []
        for j in range(tvary):
            odmocnit.append((np.dot(odozvy[j][i+1:hmoty], ramena[0:hmoty-i-1]))**2)
        
        print(f'M{i+1} = {np.sqrt(np.sum(odmocnit))/1000:.5f} kNm')

In [35]:
ohybove_momenty(odozvy, hmoty, ramena, tvary)

M1 = 6.19821 kNm
M2 = 3.60264 kNm
M3 = 1.61976 kNm
M4 = 0.00000 kNm


In [36]:
def ohybovy_moment_votknutie(momenty_preklopenia):
    M_sum = np.sum(momenty_preklopenia, axis=1)
    M_max = np.sqrt(M_sum[0]**2+M_sum[1]**2+M_sum[2]**2)/1000
    print(f'M_max = {M_max:.5f} kNm')

In [37]:
ohybovy_moment_votknutie(momenty_preklopenia)

M_max = 3.41538 kNm
